In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.5 MB/s eta 0:00:00


In [ ]:
import faiss
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel




In [ ]:
# Đường dẫn file JSON chứa văn bản pháp luật
law_data_path = "/content/drive/MyDrive/Nvidia_Final_project/Data_labeled_demo/luatdat_labeled.json"
faiss_index_path = "/content/drive/MyDrive/model_build/faiss_index"



In [ ]:
# Load dữ liệu pháp luật
with open(law_data_path, "r", encoding="utf-8") as f:
    law_data_json = json.load(f)
law_data = [item["text"] for item in law_data_json]

# Load PhoBERT
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
def encode_text(text):
    """Tạo embedding từ văn bản bằng PhoBERT"""
    inputs = phobert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    with torch.no_grad():
        outputs = phobert_model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling
    return embedding

# Tạo embeddings cho tất cả văn bản pháp luật
law_embeddings = np.array([encode_text(text) for text in law_data])



In [ ]:
# Tạo FAISS Index
embedding_dim = law_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(law_embeddings)

# Lưu FAISS index
faiss.write_index(index, faiss_index_path)
print("✅ FAISS index đã được tạo và lưu thành công!")

✅ FAISS index đã được tạo và lưu thành công!
